## AMAZON GMAIL ANALYZER
scarica titoli email 
elimina testo inutile 
fix date
splittare su piu record 

In [1]:
import imaplib
import email
import os
from datetime import datetime
import mimetypes
import pandas as pd 

In [2]:
def getMailsUsingSender(username, password, fromEmail, folderName):
    mail = imaplib.IMAP4_SSL("imap.gmail.com")
    mail.login(username, password)
    print("Login success..........")
    i=0
 
    global EMailsdf
    
    EMailsdf = pd.DataFrame(columns = ['id', 'date', 'subj','body'])
                            
    mail.select("inbox")
    # querying through search method to filter emails based on sender mail we provided.
    result, data = mail.search(None, 'FROM', '"{}"'.format(fromEmail))
    inbox_item_list = data[0].split()
    counter = 0
    for item in inbox_item_list:
        counter+=1
        result2, email_data = mail.fetch(item,'(RFC822)')
        raw_email = email_data[0][1].decode("utf-8")

        email_message = email.message_from_string(raw_email)

        to_ = email_message['To']
        from_ = email_message['From']
        subject_ = email_message['Subject']
        date_ = email_message['date']
        body_ = email_message['Body']

        to_ = "to: " + to_ + str("\n")
        from_ = "from: " + from_ + str("\n")
        date__ = "date: " + date_ + str("\n")
        subject__ = "subject: " + str(subject_) + str("\n")
        
        # Body details
        for part in email_message.walk():
            if part.get_content_type() == "text/plain":
                body = part.get_payload(decode=True)
                body_= body.decode('utf-8')
            else:
                continue
                
        EMails_record = {'id': i, 'date': str(date_), 'subj': str(subject_),'body':str(body_)}
        EMailsdf = EMailsdf.append(EMails_record, ignore_index = True)
        i=i+1

    mail.close()
    mail.logout()
    print('end')
    

In [259]:
getMailsUsingSender("fabio.bertellotti@gmail.com","cbdogiolvejhbcbq","conferma-ordine@amazon.it","tmp5")
EMailsdf.shape

Login success..........
Il tuo ordine su Amazon.it
Grazie per aver scelto Amazon.it per i tuoi acquisti. Di seguito sono visualizzate le informazioni sul tuo acquisto.

Vuoi gestire il tuo ordine online?
Per modificare il tuo ordine o controllarne lo stato, visita www.Amazon.it e clicca su "Il mio account", nella parte superiore di qualsiasi pagina.
***********************************************************
INFORMAZIONI FATTURAZIONE E SPEDIZIONE
***********************************************************
Indirizzo e-mail:	fabio.bertellotti@gmail.com
Indirizzo associato alla modalità di pagamento:

Annalisa Norcini c/o Norcini Franco
Via Cappuccini 40
Poppi, Arezzo 52014
Italia

Totale ordine: EUR 28,40

***********************************************************
	DETTAGLI ORDINE
***********************************************************

Dettagli spedizione 

l'ordine arriverà in un'unica spedizione 

***********************************************************
Numero ordine:			403-19

(209, 4)

In [260]:
EMailsdf

from datetime import datetime

for index, row in EMailsdf.iterrows():
    a = EMailsdf.at[index,'date']
    a = a[5:] #remove first chars
    a=a[:-15] #remove last chars
    date_object = datetime.strptime(a, "%d %b %Y")
    EMailsdf.at[index,'newdate'] =date_object
    EMailsdf.at[index,'year'] =int(date_object.year)

EMailsdf=EMailsdf.sort_values(by='newdate', ascending=False)
EMailsdf.groupby(['year'])["year"].count().reset_index(name="count")

,year,count
0,2012.0,1
1,2016.0,4
2,2017.0,36
3,2018.0,31
4,2019.0,29
5,2020.0,58
6,2021.0,50


In [265]:
EMailsdf = EMailsdf.reset_index(drop=True)

,id,date,subj,body,newdate,year
0,208,"Wed, 24 Nov 2021 15:55:29 +0000","Il tuo ordine Amazon.it che include ""eBuyGB El...",Amazon.it Ricezione Ordine\r\nOrdine n°408-980...,2021-11-24,2021.0
1,207,"Fri, 19 Nov 2021 12:00:53 +0000","Il tuo ordine Amazon.it che include ""Ti presen...",Amazon.it Ricezione Ordine\r\nOrdine n°408-429...,2021-11-19,2021.0
2,206,"Tue, 16 Nov 2021 20:33:26 +0000","Il tuo ordine Amazon.it che include ""Pinocchio...",Amazon.it Ricezione Ordine\r\nOrdine n°404-606...,2021-11-16,2021.0
3,205,"Mon, 8 Nov 2021 10:50:48 +0000","Il tuo ordine Amazon.it che include ""Favoto Te...",Amazon.it Ricezione Ordine\r\nOrdine n°408-794...,2021-11-08,2021.0
4,204,"Sun, 7 Nov 2021 20:34:30 +0000","Il tuo ordine Amazon.it che include ""AUTORADIO...",Amazon.it Ricezione Ordine\r\nOrdine n°404-019...,2021-11-07,2021.0
...,...,...,...,...,...,...
204,4,"Wed, 30 Nov 2016 12:05:51 +0000","Il tuo ordine Amazon.it che include ""OLYMPIA L...",Amazon.it Conferma Ordine\r\nOrdine n°405-7706...,2016-11-30,2016.0
205,3,"Thu, 24 Nov 2016 12:25:08 +0000","Il tuo ordine Amazon.it che include ""Kingston ...",Amazon.it Conferma Ordine\r\nOrdine n°405-1077...,2016-11-24,2016.0
206,2,"Mon, 10 Oct 2016 17:41:33 +0000","Il tuo ordine Amazon.it che include ""Street Fi...",Amazon.it Conferma Ordine\r\nOrdine n°171-0368...,2016-10-10,2016.0
207,1,"Mon, 26 Sep 2016 07:33:09 +0000","Il tuo ordine Amazon.it che include ""LED MICRO...",Amazon.it Conferma Ordine\r\nOrdine n°403-9591...,2016-09-26,2016.0


In [289]:

def removetextinthemiddle(frommm,tooo,text):
    try:
        first_word = frommm
        last_word = tooo
        start = int(text.find(first_word))
        res = text.rindex(last_word)
        stop = int(str(res))
        if len(text) > stop :
            text = text[0: start:] + text[stop + 1::]
    except:
      err= "#######ERR step2"
    return (text)

def QUICKREMOVETEXTINTHEMIDDLE(frommm,tooo,text):
    try:
        start=text.find(frommm) 
        stop=text.find(tooo,start) 
        text = text[:start]+text[stop:]
    except:
      err= "#######ERR stepX"
    return (text)
            
def extracttext(indexx):
    
    test_string = EMailsdf.at[indexx,'body']
    
    #remove exsternal text 
    try:
        first_word = "I tuoi articoli:"
        last_word = " Venduto da "
        start = int(test_string.find(first_word))
        res = test_string.rindex(last_word)
        stop = int(str(res))
        test_string = test_string[start:stop]
    except:
      err= "#######ERR step1"
    test_string = test_string.replace("Condizione: Nuovo\r\n ", "")
    
    #remove internal text 
    test_string=removetextinthemiddle("Totale parziale","ordine sarà spedito a",test_string)

    for x in range (6):
        test_string=QUICKREMOVETEXTINTHEMIDDLE("rdine sarà spedito a"," tuoi articoli:",test_string)
        test_string=QUICKREMOVETEXTINTHEMIDDLE("La Prenotazione al prezzo minimo","guarantee",test_string)
        test_string=QUICKREMOVETEXTINTHEMIDDLE("Venduto da","\r",test_string)
        test_string=QUICKREMOVETEXTINTHEMIDDLE("Spedizione","appena disponibili",test_string)
        test_string=QUICKREMOVETEXTINTHEMIDDLE("Totale Ordine","======",test_string)
        test_string=QUICKREMOVETEXTINTHEMIDDLE("  Arriverà:","modalità di spedizione è: ",test_string)
        test_string=QUICKREMOVETEXTINTHEMIDDLE("Gentile fabio,","access?ref_=TE_tex_gs",test_string)
            
    #remove strings
    test_string = test_string.replace("Condizione: Nuovo\r\n ", "")
    test_string = test_string.replace("guarantee", "")
    test_string = test_string.replace("(o inferiore)", "")
    test_string = test_string.replace("I tuoi articoli:\r\n", "")
    test_string = test_string.replace("tuoi articoli:", "")
    test_string = test_string.replace("Gestito da Logistica di Amazon", "")
    test_string = test_string.replace("La tua preferenza di spedizione: \r\n          Spedisci gli articoli non appena disponibili", "")
    test_string = test_string.replace("modalità di spedizione è: \r\n          Premium\r\n\r\n", "")
    test_string = test_string.replace("_______________________________________________________________________________________", "")
    test_string = test_string.replace("=======================================================================================", "")

    for x in range (6):
        test_string = test_string.replace(" \r\n\r\n", "")
    
    return(test_string)

for index, row in EMailsdf.iterrows():
    EMailsdf.at[index,'fixedtext'] =extracttext(index)
    
    
EMailsdf    

,id,date,subj,body,newdate,year,fixedtext
0,208,"Wed, 24 Nov 2021 15:55:29 +0000","Il tuo ordine Amazon.it che include ""eBuyGB El...",Amazon.it Ricezione Ordine\r\nOrdine n°408-980...,2021-11-24,2021.0,"I \r\n eBuyGB Elfi Behavin 'Mal - 30,5 cm..."
1,207,"Fri, 19 Nov 2021 12:00:53 +0000","Il tuo ordine Amazon.it che include ""Ti presen...",Amazon.it Ricezione Ordine\r\nOrdine n°408-429...,2021-11-19,2021.0,I \r\n Ti presentiamo Fire TV Stick 4K Ma...
2,206,"Tue, 16 Nov 2021 20:33:26 +0000","Il tuo ordine Amazon.it che include ""Pinocchio...",Amazon.it Ricezione Ordine\r\nOrdine n°404-606...,2021-11-16,2021.0,I \r\n CAR ROVER® Ad Alta Definizione Fot...
3,205,"Mon, 8 Nov 2021 10:50:48 +0000","Il tuo ordine Amazon.it che include ""Favoto Te...",Amazon.it Ricezione Ordine\r\nOrdine n°408-794...,2021-11-08,2021.0,I \r\n Favoto Telo Coprimoto Resistente a...
4,204,"Sun, 7 Nov 2021 20:34:30 +0000","Il tuo ordine Amazon.it che include ""AUTORADIO...",Amazon.it Ricezione Ordine\r\nOrdine n°404-019...,2021-11-07,2021.0,"I \r\n AUTORADIO per JEEP 5"" HD Screen Mu..."
...,...,...,...,...,...,...,...
204,4,"Wed, 30 Nov 2016 12:05:51 +0000","Il tuo ordine Amazon.it che include ""OLYMPIA L...",Amazon.it Conferma Ordine\r\nOrdine n°405-7706...,2016-11-30,2016.0,\r\n OLYMPIA Laminatore documenti A330 per...
205,3,"Thu, 24 Nov 2016 12:25:08 +0000","Il tuo ordine Amazon.it che include ""Kingston ...",Amazon.it Conferma Ordine\r\nOrdine n°405-1077...,2016-11-24,2016.0,\r\n Hilka 49600006 - Set di punte da legn...
206,2,"Mon, 10 Oct 2016 17:41:33 +0000","Il tuo ordine Amazon.it che include ""Street Fi...",Amazon.it Conferma Ordine\r\nOrdine n°171-0368...,2016-10-10,2016.0,\r\n Transformers Devastation - PlayStatio...
207,1,"Mon, 26 Sep 2016 07:33:09 +0000","Il tuo ordine Amazon.it che include ""LED MICRO...",Amazon.it Conferma Ordine\r\nOrdine n°403-9591...,2016-09-26,2016.0,\r\n LED MICROSCOPIO FOTOCAMERA DIGITALE U...


In [297]:

EMailsdf.columns
df=EMailsdf[['id', 'newdate', 'year', 'fixedtext']]

with pd.option_context('display.max_colwidth', None):
  display(df)

file_name = 'Amazonorders.xlsx'
df.to_excel(file_name)

,id,newdate,year,fixedtext
0,208,2021-11-24,2021.0,"I \r\n eBuyGB Elfi Behavin 'Mal - 30,5 cm Vinyl Faced Naughty Elf Doll \r\n EUR 17,99\r\n"
1,207,2021-11-19,2021.0,"I \r\n Ti presentiamo Fire TV Stick 4K Max, Wi-Fi 6, con telecomando vocale Alexa (con comandi per la TV) \r\n EUR 38,99\r\n \r\nappena disponibili\r\n\r\n L'o \r\n Amazon - Adattatore Ethernet per Fire TV \r\n EUR 14,99\r\n \r\n Amazon - Adattatore Ethernet per Fire TV \r\n EUR 14,99\r\n"
2,206,2021-11-16,2021.0,"I \r\n CAR ROVER® Ad Alta Definizione Fotocamera CCD Impermeabile Macchina Fotografica di Retrovisione con Ampio Angolo di Visione (Nero) \r\n EUR 19,99\r\n \r\n Pinocchio di Collodi, Carlo,Frezzato, Massimiliano \r\n EUR 21,99\r\n"
3,205,2021-11-08,2021.0,"I \r\n Favoto Telo Coprimoto Resistente a Acqua, Polvere, Pioggia, Vento, Escrementi di Uccelli, Copertura Antifurto per Moto Motorino Motocicletta Scooter - 245x105x125cm (XXL) \r\n EUR 21,98\r\n \r\n YOOSIDE per Garmin Venu/Vivoactive 3 cinturino, Cinturino sportivo impermeabile in silicone 20mm per Garmin Venu Sq, Vivoactive 3 Music, Vivomove HR (verde) \r\n EUR 12,99\r\n"
4,204,2021-11-07,2021.0,"I \r\n AUTORADIO per JEEP 5"" HD Screen Multifunctional Android Car Stereo con Full RCA Output& DSP per Chrysler | Jeep | Dodge(Promotion) \r\n EUR 298,00\r\n"
...,...,...,...,...
204,4,2016-11-30,2016.0,"\r\n OLYMPIA Laminatore documenti A330 per A3 inkl. 40 fogli\r\n EUR 32,99\r\n"
205,3,2016-11-24,2016.0,"\r\n Hilka 49600006 - Set di punte da legno piatte\r\n EUR 6,56\r\n Kingston CF/32GB-U2 scheda CompactFlash Ultimate 266x - 32 GB\r\n EUR 28,89\r\n JTENG Mini Micro mano Drill Bit con 10 Punte di trapano Attrezzi rotanti Mini Bohrmaschine\r\n EUR 9,99\r\n"
206,2,2016-10-10,2016.0,"\r\n Transformers Devastation - PlayStation 4\r\n EUR 17,80\r\n appena disponibili\r\n\r\n L'o Street Fighter V - Playstation 4 - Edizione Italiana\r\n EUR 27,83\r\n Street Fighter V - Playstation 4 - Edizione Italiana\r\n EUR 27,83\r\n"
207,1,2016-09-26,2016.0,"\r\n LED MICROSCOPIO FOTOCAMERA DIGITALE USB 1600x1200 + SUPPORTO\r\n EUR 13,59\r\n"


## Send quick email 

In [14]:
 # List my public Google+ activities.

import os 
import sys
print(os.getcwd())

from SendEmail import EMAIL
too= "fabio.bertellotti@gmail.com"
subjj = "Auto alert"
msgg = "warning"
EMAIL(too,subjj,msgg)


C:\Users\fabio\Desktop\PYHTON
mail sent


## tEST AREA

In [6]:
EMailsdf

,id,date,subj,body
0,0,"Mon, 13 Aug 2012 13:01:17 +0000",Il tuo ordine su Amazon.it,Grazie per aver scelto Amazon.it per i tuoi ac...
1,1,"Mon, 26 Sep 2016 07:33:09 +0000","Il tuo ordine Amazon.it che include ""LED MICRO...",Amazon.it Conferma Ordine\r\nOrdine n°403-9591...
2,2,"Mon, 10 Oct 2016 17:41:33 +0000","Il tuo ordine Amazon.it che include ""Street Fi...",Amazon.it Conferma Ordine\r\nOrdine n°171-0368...
3,3,"Thu, 24 Nov 2016 12:25:08 +0000","Il tuo ordine Amazon.it che include ""Kingston ...",Amazon.it Conferma Ordine\r\nOrdine n°405-1077...
4,4,"Wed, 30 Nov 2016 12:05:51 +0000","Il tuo ordine Amazon.it che include ""OLYMPIA L...",Amazon.it Conferma Ordine\r\nOrdine n°405-7706...
...,...,...,...,...
204,204,"Sun, 7 Nov 2021 20:34:30 +0000","Il tuo ordine Amazon.it che include ""AUTORADIO...",Amazon.it Ricezione Ordine\r\nOrdine n°404-019...
205,205,"Mon, 8 Nov 2021 10:50:48 +0000","Il tuo ordine Amazon.it che include ""Favoto Te...",Amazon.it Ricezione Ordine\r\nOrdine n°408-794...
206,206,"Tue, 16 Nov 2021 20:33:26 +0000","Il tuo ordine Amazon.it che include ""Pinocchio...",Amazon.it Ricezione Ordine\r\nOrdine n°404-606...
207,207,"Fri, 19 Nov 2021 12:00:53 +0000","Il tuo ordine Amazon.it che include ""Ti presen...",Amazon.it Ricezione Ordine\r\nOrdine n°408-429...


In [226]:
import datetime


EMailsdf.sort_values(by='date', ascending=False).head(20)

for x in range (10,20):
    a = EMailsdf.at[x,'date']

    
    a = a[5:] #remove first chars
    a=a[:-15] #remove last chars

    print(a)
    
    tmp=a.find(" ")
    
    day=a[:tmp]
    a = a[tmp:] #remove first chars
    month=a[:4][1:]
    year = a[4:][1:] #remove first chars

    x = datetime.datetime(int(year), 8, int(day))
#     print(x)
    



29 Mar 2017
5 Apr 2017
7 Apr 2017
24 Apr 2017
29 Apr 2017
6 May 2017
22 May 2017
29 May 2017
11 Jun 2017
18 Jun 2017


In [14]:
print(EMailsdf.at[204,'body'])

Amazon.it Ricezione Ordine
Ordine n°404-0191202-7931566
https://www.amazon.it/ref=TE_tex_h
_____________________________________________________________________________________

Gentile fabio,

Grazie per il tuo ordine Amazon Marketplace effettuato presso il venditore SymacShop it. Ti invieremo un'e-mail quando i tuoi articoli saranno spediti. 

Per visualizzare lo stato del tuo ordine o apportare delle modifiche, puoi consultare la sezione "I miei ordini" su Amazon.it: 
https://www.amazon.it/gp/css/your-orders-access?ref_=TE_tex_gs

Per maggiori informazioni su Amazon Marketplace, visita la nostra pagina di aiuto: 
https://www.amazon.it/marketplace?ref_=TE_tex_mp


     Arriverà:
          mercoledì, novembre 10 - 
          lunedì, novembre 15

     La tua modalità di spedizione è: 
          Spedizione standard

     L'ordine sarà spedito a: 
          fabio
          firenze, Firenze
          Italia






I tuoi articoli: 
     AUTORADIO per JEEP 5" HD Screen Multifunctional Andro

In [270]:
EMailsdf.at[8,'body']

'Amazon.it Ricezione Ordine\r\nOrdine n°404-4222690-9329134\r\nOrdine n°404-0406197-3205118\r\nhttps://www.amazon.it/ref=TE_tex_h\r\n_____________________________________________________________________________________\r\n\r\nGentile fabio,\r\n\r\nGrazie per il tuo ordine. Ti invieremo un\'e-mail quando i tuoi articoli saranno spediti. \r\nLa tua data di consegna prevista è indicata in basso. \r\nPer visualizzare lo stato del tuo ordine o apportare delle modifiche, puoi consultare la sezione "I miei ordini" su Amazon.it: \r\n    https://www.amazon.it/gp/css/your-orders-access?ref_=TE_tex_gs\r\n\r\nGli articoli acquistati sono stati suddivisi in 2 ordini. \r\n\r\n\r\n\r\n\r\n\r\n\r\n     Arriverà:\r\n          martedì, novembre 2 - \r\n          giovedì, novembre 4\r\n\r\n     La tua modalità di spedizione è: \r\n          Express\r\n\r\n     L\'ordine sarà spedito a: \r\n          fabio\r\n          firenze, Firenze\r\n          Italia\r\n\r\nI tuoi articoli: \r\n     ASICS GT-1000 10 

In [303]:
import numpy as np
dfx = pd.DataFrame({'A': [[0, 1, 2], 'foo', [], [3, 4]],'B': 1,'C': [['a', 'b', 'c'], np.nan, [], ['d', 'e']]})
dfx


,A,B,C
0,"[0, 1, 2]",1,"[a, b, c]"
1,foo,1,NaN
2,[],1,[]
3,"[3, 4]",1,"[d, e]"


In [304]:
dfx.explode('A')

,A,B,C
0,0,1,"[a, b, c]"
0,1,1,"[a, b, c]"
0,2,1,"[a, b, c]"
1,foo,1,NaN
2,NaN,1,[]
3,3,1,"[d, e]"
3,4,1,"[d, e]"


In [31]:

def explode(df, lst_cols, fill_value='', preserve_index=False):
    # make sure `lst_cols` is list-alike
    if (lst_cols is not None
        and len(lst_cols) > 0
        and not isinstance(lst_cols, (list, tuple, np.ndarray, pd.Series))):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)
    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()
    # preserve original index values    
    idx = np.repeat(df.index.values, lens)
    # create "exploded" DF
    res = (pd.DataFrame({
                col:np.repeat(df[col].values, lens)
                for col in idx_cols},
                index=idx)
             .assign(**{col:np.concatenate(df.loc[lens>0, col].values)
                            for col in lst_cols}))
    # append those rows that have empty lists
    if (lens == 0).any():
        # at least one list in cells is empty
        res = (res.append(df.loc[lens==0, idx_cols], sort=False)
                  .fillna(fill_value))
    # revert the original index order
    res = res.sort_index()
    # reset index if requested
    if not preserve_index:        
        res = res.reset_index(drop=True)
    return res